In [1]:
import pandas as pd
import torch
from model.FieldingModel import FieldingModel


/home/wahoo/.pyenv/versions/3.13.2/envs/ootp-analytics/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
league =  'BZ'
season_start = 2016
season_end = 2017
positions = [9,8,7,6,5,4,3,2]
ratings_type = "Standard"

In [ ]:

    
for position in positions:
    print(f"\r\nTraining model for position {position}")
    fieldingModel = FieldingModel(league, season_start, season_end, position, ratings_type)
    fieldingModel.load_data(position)

    epoch_count, loss = fieldingModel.train(5000)
    print(f"Epoch [{epoch_count}/{5000}], Loss: {loss:.7f}")

    test_loss = fieldingModel.evaluate()
    print(f"Test Mean Squared Error: {test_loss:.7f}")

    feature_importance = fieldingModel.feature_importance()
    print(feature_importance)

    fieldingModel.save_model()


Training model for position 9


NameError: name 'season' is not defined